In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sklearn
import glob
import pickle
import numpy as np
import copy
import random
import os
import pandas as pd
import re
import timeit

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV 


from sklearn import model_selection

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

from nltk import sent_tokenize

from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score

from helpers import LemmaTokenizer, StemTokenizer, SnowballStemTokenizer, misc_features, cleaner, get_tfidf_vectorizer

import gensim
import gensim.parsing.preprocessing as gpp

import matplotlib.pyplot as plt

In [3]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 10)

### loading files

In [4]:
try:
    cwd = os.path.dirname(os.path.abspath(__file__))
except:
    cwd = os.getcwd()

In [5]:
data_folder = os.path.join(cwd, "data/")

In [6]:
train_filepath = os.path.join(cwd, "data", "train.csv")

In [7]:
train_perc = 0.8

train_df = pd.read_csv(train_filepath)
X_df = train_df.drop(columns=['is_duplicate'])
Y_df = train_df['is_duplicate']

X_train, X_test, y_train, y_test =  train_test_split(X_df, Y_df, test_size=0.2, random_state=42, stratify=Y_df)
X_val, X_test, y_val, y_test =  train_test_split(X_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

In [8]:
# X_test = X_val
# y_test = y_val

In [9]:
X_train.head(5)

,id,qid1,qid2,question1,question2
341317,341317,469121,469122,What is an anagram?,Is there an anagram for anagram?
237560,237560,83580,348749,Social Media: What does it mean when a guy del...,My boyfriend treasures memories he had with hi...
280719,280719,400352,400353,Is this a good time to buy Apple stock?,Is it a good time to buy Apple stock at $92?
288940,288940,409976,409977,What are the consequences of getting high on h...,What is the number of Lok Sabha constituencies...
27093,27093,50355,50356,What are some good mixes with watermelon vodka?,What's a good mixer for watermelon vodka?


In [10]:
# combining the strings together

### word overlaps

In [9]:
stop_words = set(stopwords.words("english"))

def get_word_diffs(questions):
    q1_list = str(questions['question1']).lower().split()
    q2_list = str(questions['question2']).lower().split()
    
    q1_list = [word for word in q1_list if word not in stop_words]
    q2_list = [word for word in q2_list if word not in stop_words]
    
    diff = abs(len(q1_list) - len(q2_list))
    
    return diff

def get_word_overlaps(questions):
#     print(questions)
    q1_list = str(questions['question1']).lower().split()
    q2_list = str(questions['question2']).lower().split()
    
    q1_list = [word for word in q1_list if word not in stop_words]
    q2_list = [word for word in q2_list if word not in stop_words]
    
    q1_set = set(q1_list)
    q2_set = set(q2_list)
    q12_set = set(q1_list + q2_list)
    
    overlap_count = q1_set.intersection(q2_set)
    overlap_perc = float(len(overlap_count)) / len(q12_set)
    
    return overlap_perc

### word embeddings

In [11]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = "/Users/nahiyanmalik/Downloads/glove.840B.300d.txt"
tmp_file = get_tmpfile("test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)

model = KeyedVectors.load_word2vec_format(tmp_file)

KeyboardInterrupt: 

### preprocessing

In [10]:
filts_to_use = [gpp.strip_tags, gpp.strip_multiple_whitespaces, gpp.remove_stopwords, gpp.strip_punctuation, gpp.stem_text]
# filts_to_use = [gpp.strip_tags, gpp.strip_multiple_whitespaces, gpp.remove_stopwords]

def get_question_tokens(dataset):       
    if dataset == 'train':
        X_temp = X_train
    elif dataset == 'val':
        X_temp = X_val
    else:
        X_temp = X_test
        
    qs = pd.Series(pd.concat([X_temp['question1'], X_temp['question2']]), dtype=str)
    qs.dropna()
    
    for q in qs:
        yield gpp.preprocess_string(q, filts_to_use)        

In [11]:
def get_vector_tokens(wv_model, dataset):
    for q in get_question_tokens(dataset):
#         print(q)
        tfidf_tokens = []
        for tok in q:
#             print(tok)
            try:
#                 print(tok)
                vec = wv_model.wv[tok]
                tfidf_tokens.append(tok)
            except:
#                 print("something wrong", tok)
                continue
        yield np.array(tfidf_tokens)

In [12]:
def get_weights_vectors(tfidf_mtx, tfidf_vectorizer, wv_model):
    weights, w2v = [], []
    rows = tfidf_mtx.shape[0]
    inverse_vocab_dict = {v: k for k, v in tfidf_vectorizer.vocabulary_.items()}
    
    for doc in range(rows):
        features = tfidf_mtx[doc,:].nonzero()[1]
        weights.append(np.array([tfidf_mtx[doc, x] for x in features]))
        w2v.append(np.array([wv_model.wv[inverse_vocab_dict[x]] for x in features]))
    
    return np.array(weights), np.array(w2v)

### tfidf weighted word embeddings

In [17]:
tfidf_transformer = TfidfVectorizer(analyzer=lambda x:x)

X_tfidf = tfidf_transformer.fit_transform(get_vector_tokens(model, 'train'))
X_q1_tfidf = X_tfidf[:len(X_train)]
X_q2_tfidf = X_tfidf[len(X_train):]

/Users/nahiyanmalik/anaconda/envs/mlp3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


In [18]:
X1_w, X1 = get_weights_vectors(X_q1_tfidf, tfidf_transformer, model)
X2_w, X2 = get_weights_vectors(X_q2_tfidf, tfidf_transformer, model)

/Users/nahiyanmalik/anaconda/envs/mlp3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


In [19]:
pickle.dump(X1_w, open(data_folder+'X1_w.p','wb'))
pickle.dump(X2_w, open(data_folder+'X2_w.p','wb'))
pickle.dump(X1, open(data_folder+'X1.p','wb'))
pickle.dump(X2, open(data_folder+'X2.p','wb'))

In [22]:
X_test_tfidf = tfidf_transformer.transform(get_vector_tokens(model, 'test'))
X_test_q1_tfidf = X_test_tfidf[:len(X_test)]
X_test_q2_tfidf = X_test_tfidf[len(X_test):]

X1_test_w, X1_test = get_weights_vectors(X_test_q1_tfidf, tfidf_transformer, model)
X2_test_w, X2_test = get_weights_vectors(X_test_q2_tfidf, tfidf_transformer, model)

/Users/nahiyanmalik/anaconda/envs/mlp3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


In [23]:
pickle.dump(X1_test_w, open(data_folder+'X1_test_w.p','wb'))
pickle.dump(X2_test_w, open(data_folder+'X2_test_w.p','wb'))
pickle.dump(X1_test, open(data_folder+'X1_test.p','wb'))
pickle.dump(X2_test, open(data_folder+'X2_test.p','wb'))

In [14]:
# after dumping for first time

In [13]:
X1_w = pickle.load(open(data_folder+'X1_w.p','rb'))
X2_w = pickle.load(open(data_folder+'X2_w.p','rb'))
X1 = pickle.load(open(data_folder+'X1.p','rb'))
X2 = pickle.load(open(data_folder+'X2.p','rb'))

In [14]:
X1_test_w = pickle.load(open(data_folder+'X1_test_w.p','rb'))
X2_test_w = pickle.load(open(data_folder+'X2_test_w.p','rb'))
X1_test = pickle.load(open(data_folder+'X1_test.p','rb'))
X2_test = pickle.load(open(data_folder+'X2_test.p','rb'))

In [17]:
# https://www.researchgate.net/publication/228560462_A_Metric_for_Paraphrase_Detection
# euclidean distances

In [18]:
from sklearn.metrics import pairwise_distances

In [19]:
# taken from https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html

distance_metrics = ['cityblock', 'cosine', 'euclidean', 
                    'l1', 'l2', 'manhattan', 'braycurtis', 
                    'canberra', 'chebyshev', 'correlation', 
                    'dice', 'hamming', 'jaccard', 'kulsinski', 
#                     'mahalanobis', 
                    'minkowski', 'rogerstanimoto', 
                    'russellrao', 'seuclidean', 'sokalmichener', 
                    'sokalsneath', 'sqeuclidean', 'yule']

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
def get_w_means(comp, w):
    means = []
    for idx in range(len(comp)):
        try:
            mean = np.average(comp[idx], axis=0, weights=w[idx])
            means.append(mean)
        except:
            means.append(0.0)
    
    return means

In [16]:
def get_pairwise_dist(comp1, comp2, weight1, weight2, method):
    dists = []
    for idx in range(len(comp1)):
        try:
            dist_mtx = pairwise_distances(comp1[idx], comp2[idx], metric=method) 
            dist = np.average(dist_mtx, weights=np.matmul(weight1[idx].reshape(-1,1),weight2[idx].reshape(-1,1).T))
            dists.append(dist)
        except:
            dists.append(0)
        
    return dists

In [17]:
def get_all_distances(comp1, comp2, weight1, weight2, distance_metrics=distance_metrics):
    dist_dict = {}
    for metric in distance_metrics:
        print(metric)
        dists = get_pairwise_dist(comp1, comp2, weight1, weight2, metric)
        dist_dict[metric] = dists
        pickle.dump(dists, open(data_folder + metric + '_' + 'train_dists.p','wb'))
        
    return dist_dict

NameError: name 'distance_metrics' is not defined

In [23]:
train_dists = get_all_distances(X1[:], X2[:], X1_w[:], X2_w[:])

cityblock
cosine
euclidean
l1
l2
manhattan
braycurtis
canberra
chebyshev
correlation
dice
hamming
jaccard
kulsinski
minkowski
rogerstanimoto
russellrao
seuclidean
sokalmichener
sokalsneath
sqeuclidean
yule


In [24]:
# pickle.dump(train_dists, open(data_folder+'train_dists.p','wb'))

In [ ]:
# test_dists = get_all_distances(X1_test[:], X2_test[:], X1_test_w[:], X2_test_w[:])

In [ ]:
# pickle.dump(test_dists, open(data_folder+'test_dists.p','wb'))

In [26]:
train_dists = pickle.load(open(data_folder+'train_dists.p','rb'))
test_dists = pickle.load(open(data_folder+'test_dists.p','rb'))

In [38]:
train_X1_wmeans = get_w_means(X1, X1_w[:])

In [39]:
train_X2_wmeans = get_w_means(X2, X2_w[:])

In [40]:
train_wmeans_dict = {
    'X1_wmean': train_X1_wmeans,
    'x2_wmean': train_X2_wmeans
}

In [18]:
# pickle.dump(train_wmeans_dict, open(data_folder+'train_wmeans.p','wb'))
train_wmeans_dict = pickle.load(open(data_folder+'train_wmeans.p','rb'))

In [17]:
test_X1_wmeans = get_w_means(X1_test, X1_test_w[:])
test_X2_wmeans = get_w_means(X2_test, X2_test_w[:])

In [18]:
test_wmeans_dict = {
    'X1_test_wmean': test_X1_wmeans,
    'x2_test_wmean': test_X2_wmeans
}

In [19]:
# pickle.dump(test_wmeans_dict, open(data_folder+'test_wmeans.p','wb'))
test_wmeans_dict = pickle.load(open(data_folder+'test_wmeans.p','rb'))

### now getting approximate string matching using fuzzywuzzy

In [27]:
from fuzzywuzzy import fuzz

In [28]:
get_rel_size_diff = lambda row: abs(len(str(row['question1'])) - len(str(row['question2'])))
get_fz_ratio = lambda row: fuzz.ratio(str(row['question1']), str(row['question2']))
get_fz_partial_ratio = lambda row: fuzz.partial_ratio(str(row['question1']), str(row['question2']))
get_fz_token_sort_ratio = lambda row: fuzz.token_sort_ratio(str(row['question1']), str(row['question2']))
get_fz_token_set_ratio = lambda row: fuzz.token_set_ratio(str(row['question1']), str(row['question2']))

def get_fz_metric(X, fn):
    print(fn)
    fz_met = X.apply(fn, axis=1)
    
    return fz_met

In [56]:
train_fz_dict = {
    'rel_size_diff' : list(get_fz_metric(X_train, get_rel_size_diff)),
    'fz_ratio' : list(get_fz_metric(X_train, get_fz_ratio)),
    'fz_partial_ratio' : list(get_fz_metric(X_train, get_fz_partial_ratio)),
    'fz_token_sort_ratio' : list(get_fz_metric(X_train, get_fz_token_sort_ratio)),
    'fz_token_set_ratio' : list(get_fz_metric(X_train, get_fz_token_set_ratio))
}

<function <lambda> at 0x107477f28>
<function <lambda> at 0x1a419ce6a8>
<function <lambda> at 0x1a419ce598>
<function <lambda> at 0x1a419ce488>
<function <lambda> at 0x1a419ce510>


In [22]:
# pickle.dump(train_fz_dict, open(data_folder+'train_fz.p','wb'))
train_fz_dict = pickle.load(open(data_folder+'train_fz.p','rb'))

In [23]:
test_fz_dict = {
    'rel_size_diff' : list(get_fz_metric(X_test, get_rel_size_diff)),
    'fz_ratio' : list(get_fz_metric(X_test, get_fz_ratio)),
    'fz_partial_ratio' : list(get_fz_metric(X_test, get_fz_partial_ratio)),
    'fz_token_sort_ratio' : list(get_fz_metric(X_test, get_fz_token_sort_ratio)),
    'fz_token_set_ratio' : list(get_fz_metric(X_test, get_fz_token_set_ratio))
}

<function <lambda> at 0x1a20a2df28>
<function <lambda> at 0x1a1f71fb70>
<function <lambda> at 0x1a2d1a5510>
<function <lambda> at 0x1a2d1a5598>
<function <lambda> at 0x1a2d1a5620>


In [23]:
# pickle.dump(test_fz_dict, open(data_folder+'test_fz.p','wb'))
test_fz_dict = pickle.load(open(data_folder+'test_fz.p','rb'))

## model time

In [29]:
X_train_features_dict = {}

for k in train_dists.keys():
    X_train_features_dict[k] = train_dists[k]
    
for k in train_wmeans_dict.keys():
    X_train_features_dict[k] = [np.mean(i) for i in train_wmeans_dict[k]]
    
for k in train_fz_dict.keys():
    X_train_features_dict[k] = train_fz_dict[k]

In [30]:
X_train_features_df = pd.DataFrame(X_train_features_dict)

In [31]:
X_train_features_df = X_train_features_df.fillna(0)

In [34]:
# X_train_features_df = X_train_features_df.drop(columns=['dice', 'jaccard', 'kulsinski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'yule'])

In [32]:
X_test_features_dict = {}

for k in test_dists.keys():
    X_test_features_dict[k] = test_dists[k]
    
for k in test_wmeans_dict.keys():
    X_test_features_dict[k] = [np.mean(i) for i in test_wmeans_dict[k]]
    
for k in test_fz_dict.keys():
    X_test_features_dict[k] = test_fz_dict[k]

In [33]:
X_test_features_df = pd.DataFrame(X_test_features_dict)
X_test_features_df = X_test_features_df.fillna(0)

In [27]:
# X_test_features_df = X_test_features_df.drop(columns=['dice', 'jaccard', 'kulsinski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'yule'])

In [28]:
# standard scalar

In [34]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_features_scaled_df = scaler.fit_transform(X_train_features_df)
X_test_features_scaled_df = scaler.transform(X_test_features_df)


In [87]:
# logistic regression

In [28]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

log_reg = LogisticRegression(verbose=10)
params = {'C': [1, 10, 100, 1000],
        'tol': [0, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]
       }

log_reg_cv = RandomizedSearchCV(log_reg, param_distributions=params, cv=5, n_jobs=-1)
log_reg_cv.fit(X_train_features_scaled_df, y_train)

/Users/nahiyanmalik/anaconda/envs/mlp3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[LibLinear]

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='warn', n_jobs=None,
                                                penalty='l2', random_state=None,
                                                solver='warn', tol=0.0001,
                                                verbose=10, warm_start=False),
                   iid='warn', n_iter=10, n_jobs=-1,
                   param_distributions={'C': [1, 10, 100, 1000],
                                        'tol': [0, 0.1, 0.01, 0.001, 0.0001,
                                                1e-05, 1e-06]},
                   pre_dispatch='2*n_jobs', random_state=None, refi

In [29]:
log_reg_cv.best_params_


{'tol': 0.1, 'C': 10}

In [36]:
log_reg = LogisticRegression(tol=0.1, C=100, max_iter=10)

log_reg.fit(X_train_features_scaled_df, y_train)
y_pred = log_reg.predict(X_test_features_scaled_df)

accuracy = accuracy_score(y_pred, y_test)
print(accuracy)

/Users/nahiyanmalik/anaconda/envs/mlp3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6641519701204581


In [119]:
# svm

In [37]:
svm_ins = LinearSVC()

svm_ins.fit(X_train_features_scaled_df, y_train)
y_pred = svm_ins.predict(X_test_features_scaled_df)

accuracy = accuracy_score(y_pred, y_test)
print(accuracy)

0.6638551534789384


/Users/nahiyanmalik/anaconda/envs/mlp3/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [35]:
# nn

In [38]:
nn = MLPClassifier(verbose=10, early_stopping=True, solver='adam', learning_rate='adaptive')

nn.fit(X_train_features_scaled_df, y_train)
y_pred = nn.predict(X_test_features_scaled_df)

accuracy = accuracy_score(y_pred, y_test)
print("accuracy:", accuracy)

Iteration 1, loss = 0.51928016
Validation score: 0.714290
Iteration 2, loss = 0.50584778
Validation score: 0.716825
Iteration 3, loss = 0.50192965
Validation score: 0.723596
Iteration 4, loss = 0.49947510
Validation score: 0.725049
Iteration 5, loss = 0.49754364
Validation score: 0.723782
Iteration 6, loss = 0.49637481
Validation score: 0.727399
Iteration 7, loss = 0.49516857
Validation score: 0.728327
Iteration 8, loss = 0.49435561
Validation score: 0.729996
Iteration 9, loss = 0.49371520
Validation score: 0.730893
Iteration 10, loss = 0.49307646
Validation score: 0.727677
Iteration 11, loss = 0.49255752
Validation score: 0.729533
Iteration 12, loss = 0.49195110
Validation score: 0.729533
Iteration 13, loss = 0.49144086
Validation score: 0.730429
Iteration 14, loss = 0.49119794
Validation score: 0.732933
Iteration 15, loss = 0.49049823
Validation score: 0.730305
Iteration 16, loss = 0.49010078
Validation score: 0.730182
Iteration 17, loss = 0.48989748
Validation score: 0.728265
Iterat

In [41]:
import xgboost as xgb

In [40]:
params_xg = {'n_estimators' : [1, 2, 4, 8, 16, 32, 64, 100, 200, 500, 1000],
               'gamma': [0, 0.001, .01, 1, 10], 
               'learning_rate' : [0.001, 0.01, 0.1, 1, 10],
               'max_depth' : [1, 5, 6, 10, 20]
                 }

In [42]:
xgb_ins = xgb.XGBClassifier()

In [42]:
xgb_cv = RandomizedSearchCV(xgb_ins, param_distributions=params_xg, cv=5, n_jobs=4)
xgb_cv.fit(X_train_features_scaled_df, y_train)

KeyboardInterrupt: 

In [43]:
xgb_ins = xgb.XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=5, verbosity=1, n_jobs=4)

xgb_ins.fit(X_train_features_scaled_df, y_train)
y_pred = xgb_ins.predict(X_test_features_scaled_df)

accuracy = accuracy_score(y_pred, y_test)
print("accuracy:", accuracy)

accuracy: 0.7206708056098345


In [44]:
xgb_ins = xgb.XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=10, verbosity=1, n_jobs=4)

xgb_ins.fit(X_train_features_scaled_df, y_train)
y_pred = xgb_ins.predict(X_test_features_scaled_df)

accuracy = accuracy_score(y_pred, y_test)
print("accuracy:", accuracy)

accuracy: 0.7273244453239012


In [46]:
xgb_ins = xgb.XGBClassifier(learning_rate=0.1, n_estimators=1000, max_depth=10, verbosity=1, n_jobs=4)

xgb_ins.fit(X_train_features_scaled_df, y_train)
y_pred = xgb_ins.predict(X_test_features_scaled_df)

accuracy = accuracy_score(y_pred, y_test)
print("accuracy:", accuracy)

accuracy: 0.7251477899527566


In [44]:
# random forest

In [39]:
rfc = RandomForestClassifier(n_estimators=1000, verbose=10, n_jobs=4)

rfc.fit(X_train_features_scaled_df, y_train)
y_pred = rfc.predict(X_test_features_scaled_df)

accuracy = accuracy_score(y_pred, y_test)
print("accuracy:", accuracy)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000building tree 4 of 1000

building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000


[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    3.8s


building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000


[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    5.7s


...

building tree 922 of 1000
building tree 923 of 1000
building tree 924 of 1000
building tree 925 of 1000
building tree 926 of 1000
building tree 927 of 1000
building tree 928 of 1000
building tree 929 of 1000
building tree 930 of 1000
building tree 931 of 1000
building tree 932 of 1000
building tree 933 of 1000
building tree 934 of 1000
building tree 935 of 1000
building tree 936 of 1000
building tree 937 of 1000
building tree 938 of 1000
building tree 939 of 1000
building tree 940 of 1000
building tree 941 of 1000
building tree 942 of 1000
building tree 943 of 1000
building tree 944 of 1000
building tree 945 of 1000
building tree 946 of 1000
building tree 947 of 1000
building tree 948 of 1000
building tree 949 of 1000
building tree 950 of 1000
building tree 951 of 1000
building tree 952 of 1000
building tree 953 of 1000
building tree 954 of 1000
building tree 955 of 1000
building tree 956 of 1000
building tree 957 of 1000
building tree 958 of 1000
building tree 959 of 1000
building tre

[Parallel(n_jobs=4)]: Done 960 tasks      | elapsed:  8.0min


building tree 965 of 1000
building tree 966 of 1000
building tree 967 of 1000
building tree 968 of 1000
building tree 969 of 1000
building tree 970 of 1000
building tree 971 of 1000
building tree 972 of 1000
building tree 973 of 1000
building tree 974 of 1000
building tree 975 of 1000
building tree 976 of 1000
building tree 977 of 1000
building tree 978 of 1000building tree 979 of 1000

building tree 980 of 1000
building tree 981 of 1000
building tree 982 of 1000
building tree 983 of 1000
building tree 984 of 1000
building tree 985 of 1000
building tree 986 of 1000
building tree 987 of 1000
building tree 988 of 1000
building tree 989 of 1000
building tree 990 of 1000
building tree 991 of 1000
building tree 992 of 1000
building tree 993 of 1000
building tree 994 of 1000
building tree 995 of 1000
building tree 996 of 1000
building tree 997 of 1000
building tree 998 of 1000
building tree 999 of 1000
building tree 1000 of 1000


[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed: 15.3min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done 137 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:   

accuracy: 0.7335823295159415
